In [31]:
import pydriller
import pandas as pd
import matplotlib.pyplot as plt
import os
import regex as re
import mccabe
import seaborn as sns
import json

# Repository Setup and Procedure for React Codebase Analysis

To keep our work organized and avoid modifying the React repository directly, we set up our own repository with React included in our `.gitignore` file (`/react`). This allows us to access the React codebase for analysis without committing any changes to it. We prepared for the assignment using the following steps:

1. Clone the React codebase within our project directory:
   ```bash
   git clone https://github.com/facebook/react
   ```

2. Check out the required React version (v18.3.1) by navigating into the `react` directory and switching branches:
   ```bash
   cd react
   git checkout v18.3.1
   cd ..
   ```

By following this setup, we maintain a clean separation between our analysis files and the React codebase, ensuring that React remains unmodified in our own repository.

# Task 1: Component-Level Codebase Overview

- Using the latest stable release of React, v18.3.1, list all the components. In
React, there are two main types of components: Class-based Components and
Functional Components. Class-based components are defined where there is
a class declaration that extends ‘React.Component‘ or ‘React.PureComponent‘.
Function-based components are JavaScript functions that return JSX elements
(JavaScript XML, return elements enclosed in brackets (return <...>) ). To
find these components in the React codebase, look for any ‘.js‘ file, and scan the
file content for patterns mentioned above (hint: use regrex to find the patterns)

In [32]:
def list_react_components(directory='./react'):
    class_components = []
    function_components = []
    
    # Regular expressions to match components
    class_component_pattern = re.compile(r'class\s+(\w+)\s+extends\s+React\.(Component|PureComponent)')
    function_component_pattern = re.compile(r'function\s+(\w+)\s*\(.*\)\s*\{[^}]*return\s*\(<')

    # Walk through the directory and analyze .js files
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.js'):
                file_path = os.path.join(root, file)
                
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    
                    # Find class components
                    class_matches = class_component_pattern.findall(content)
                    for match in class_matches:
                        component_name = match[0]
                        class_components.append((component_name, file_path))
                    
                    # Find function components
                    function_matches = function_component_pattern.findall(content)
                    for match in function_matches:
                        component_name = match
                        function_components.append((component_name, file_path))

    # Print results
    print("Class-based Components:")
    for component, path in class_components:
        print(f"Component: {component}, File: {path}")
    
    print("\nFunction-based Components:")
    for component, path in function_components:
        print(f"Component: {component}, File: {path}")

    return class_components, function_components

# Run the function
class_components, function_components = list_react_components()


Class-based Components:
Component: ReactImage0, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: AbstractLink1, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: Link2, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: AbstractButton3, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: XUIButton4, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: AbstractPopoverButton5, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: ReactXUIPopoverButton6, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: BIGAdAccountSelector7, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: FluxContainer_AdsPEBIGAdAccountSelectorContainer_8, File: ./react/scripts/bench/benchmarks/pe-class-components/benchmark.js
Component: ErrorBoundary9, File: ./react/s

---

- Use Madge tool<sup>3</sup> to detect dependencies between React files. Madge is a tool
that analyzes dependencies in software projects making it easier to understand
the complexities of file interactions within a codebase. Report all the dependen-
cies in a JSON file. Use this JSON file and find and report the top 3 files with
the highest number of dependencies. Provide the data in a JSON file with the
following format:

In [33]:
os.makedirs('./documentation', exist_ok=True)

```bash
npm install -g madge
cd react
madge --json . > ../documentation/dependencies_18.3.1.json
```

In [34]:
def analyze_dependencies(version):
    input_file=(f'documentation/dependencies_{version}.json')
    output_file= (f'documentation/top_dependencies_{version}.json')
    # Load the dependencies data
    with open(input_file, 'r') as f:
        dependencies = json.load(f)
    
    # Count dependencies for each file
    dependency_counts = {file: len(dependencies[file]) for file in dependencies}
    
    # Find the top 3 files with the highest number of dependencies
    top_files = sorted(dependency_counts.items(), key=lambda x: x[1], reverse=True)[:3]
    
    # Format the results
    top_dependencies = {file: dependencies[file] for file, _ in top_files}
    
    # Save the top dependencies to a JSON file
    with open(output_file, 'w') as f:
        json.dump(top_dependencies, f, indent=4)
    
    print(f"Top dependencies saved to {output_file}")

In [35]:
# Run the analysis
version="18.3.1"
analyze_dependencies(version)

Top dependencies saved to documentation/top_dependencies_18.3.1.json


---

- Identify the commit that resulted in the most substantial change between v17.0.1
and v17.0.2 versions of React. Use Git commands to explore the commit history
and quantify changes based on the number of files affected. Document the
commit hash, number of files changed, number of insertions and number of
deletions.

```bash
cd react
git fetch --all
git checkout v17.0.2
git log v17.0.1..v17.0.2 --stat
```

**Most Substantial Commit Between v17.0.1. and v17.0.2**  
- Commit Hash: `12adaffef7105e2714f82651ea51936c563fe15c`  
- Number of Files Changed: 4  
- Number of Insertions: 15  
- Number of Deletions: 123

<pre>
<span style="color: yellow;">commit 12adaffef7105e2714f82651ea51936c563fe15c (<span style="color: cyan;">HEAD</span>, tag: v17.0.2, <span style="color: red;">origin/17.0.2</span>)</span>
Author: Brian Vaughn &lt;bvaughn@fb.com&gt;
Date:   Thu Feb 18 11:21:52 2021 -0500

    Remove scheduler sampling profiler shared array buffer (#20840)
    
    No one has been using this data so there's no reason to collect it. Event log has been maintained and tests have been updated.

 packages/scheduler/src/Scheduler.js                         |  2 <span style="color: red;">--</span>
 packages/scheduler/src/SchedulerFeatureFlags.js             |  2 <span style="color: green;">+-</span>
 packages/scheduler/src/SchedulerProfiling.js                | 56 <span style="color: red;">--------------------------------------------------------</span>
 packages/scheduler/src/__tests__/SchedulerProfiling-test.js | 78 <span style="color: green;">+++++++++++++</span><span style="color: red;">----------------------------------------------------------------</span>
 4 files changed, 15 insertions(+), 123 deletions(-)
</pre>

---

- Check out the repository at the commit hash identified in the previous step. This allows you to view the state of the codebase at the time of that commit. After checking out, use Madge to analyze the dependencies at this specific commit point. To trace how dependencies changed and evolved over time compare these dependencies to those detected before the checkout (i.e., in the current latest version). Document any new dependencies introduced, or dependencies removed.

```bash
git checkout b2bbee7ba31bb7d212a9ff2e682a695a32f8a87f
madge --json . > ../documentation/dependencies_b2bbee7ba31bb7d212a9ff2e682a695a32f8a87f.json
```

In [38]:
def compare_dependencies(version_one, version_two):
    file_one=(f'documentation/dependencies_{version_one}.json')
    file_two=(f'documentation/dependencies_{version_two}.json')
    output_file=(f'documentation/dependenies_changed_{version_one}_{version_two}.json')
    # Load the dependency files
    with open(file_one, 'r') as f:
        file_one_dependency = json.load(f)
    
    with open(file_two, 'r') as f:
        file_two_dependency = json.load(f)
    
    # Compare dependencies
    introduced = {}
    removed = {}
    
    # Check for new dependencies
    for file, dependency in file_two_dependency.items():
        if file not in file_one_dependency:
            introduced[file] = dependency
        else:
            new_dependency = set(dependency) - set(file_one_dependency[file])
            if new_dependency:
                introduced[file] = list(new_dependency)
    
    # Check for removed dependencies
    for file, dependency in file_one_dependency.items():
        if file not in file_two_dependency:
            removed[file] = dependency
        else:
            removed_dependency = set(dependency) - set(file_two_dependency[file])
            if removed_dependency:
                removed[file] = list(removed_dependency)
    
    # Save the results
    changes = {
        "introduced": introduced,
        "removed": removed
    }
    
    with open(output_file, 'w') as f:
        json.dump(changes, f, indent=4)
    
    print(f"Dependency changes saved to {output_file}")

In [39]:
latest_version='18.3.1'
commit_version='b2bbee7ba31bb7d212a9ff2e682a695a32f8a87f'
# Run the comparison
compare_dependencies(commit_version, latest_version)

Dependency changes saved to documentation/dependenies_changed_b2bbee7ba31bb7d212a9ff2e682a695a32f8a87f_18.3.1.json


---

# Task 2: Temporal and Logical Coupling Analysis